Jupyter Notebook for dataset preprocessing for 'Dreams' dataset. 

In [2]:
import pandas as pd
dream = pd.read_csv('dreams_dataset.csv')
dream

,dream_id,dreamer,description,dream_date,dream_language,text_dream,characters_code,emotions_code,aggression_code,friendliness_code,...,Male,Animal,Friends,Family,Dead&Imaginary,Aggression/Friendliness,A/CIndex,F/CIndex,S/CIndex,NegativeEmotions
0,1,alta,Alta: a detailed dreamer,1957,en,"The one at the Meads's house, where it's bigge...","2ISA, 1MKA, 1FDA, 1IOA, 2ISA",NaN,2IKA > Q,2IKA 4> Q,...,0.500000,0.000000,0.200000,0.200000,0.0,0.000,0.200000,0.200000,0.0,0.0
1,2,alta,Alta: a detailed dreamer,8/11/1967,en,I'm at a family reunion in a large fine house ...,"2ISA, people, 2ISA",SD 2IKA,"D > Q, Q > 2ISA",NaN,...,0.000000,0.000000,0.000000,0.000000,0.0,1.000,0.666667,0.000000,0.0,1.0
2,3,alta,Alta: a detailed dreamer,8/1/1985,en,I watch a plane fly past and shortly realize i...,"2ISA, 2ISA, 1FSA, 1MBA, 1IOA, 2ISA, 2FDA","SD 1ISA, AP D, AP D","It PRP >, It PRP >, D > 1FKA",NaN,...,0.333333,0.000000,0.000000,0.285714,0.0,1.000,0.428571,0.000000,0.0,1.0
3,4,alta,Alta: a detailed dreamer,1985?,en,Me pulling the green leaves and berries off so...,"1MAA, 1FMA, 2ISA, 2IKA, 1ANI, 1ANI, 1IOA, 2ISA...","SD 2ISA, SD D","Q > Q, 2ISA > Q, 2ISA > Q, D > 1MSA","1IKA 4> Q, 2ISA 4> 2ISA",...,0.666667,0.176471,0.142857,0.142857,0.0,1.000,0.235294,0.117647,0.0,1.0
4,5,alta,Alta: a detailed dreamer,1985?,en,I'm in a room that reminds me of (but definite...,"1IRA, 1MSA, 1ISA, 2ISA, 1ISA, 1IKA","AP D, AP D, AP 1MSA, CO D, SD D, AP D","1MSA > D, Q > Q, D > 2IKA, D > 2IKA, D > 1MSA,...",D 4> Q,...,1.000000,0.000000,0.166667,0.166667,0.0,0.875,1.333333,0.166667,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20995,33062,west_coast_teens,West Coast teenage girls,"F, age 18",en,The dream was about me and my boyfriend going ...,"2ISA, 2ISA, 1ISA","HA D, AP D","2IKA > Q, D > D",NaN,...,0.000000,0.000000,0.000000,0.000000,0.0,1.000,0.666667,0.000000,0.0,0.5
20996,33063,west_coast_teens,West Coast teenage girls,"F, age 18",en,Two weeks ago this guy asked me to Senior Ball...,1ISA,HA D,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,1.0,0.0
20997,33064,west_coast_teens,West Coast teenage girls,"F, age 18",en,My boyfriend just broke up with me so he was o...,"1ISA, 1FSA",AP D,Q > D,NaN,...,0.000000,0.000000,0.000000,0.000000,0.0,1.000,0.500000,0.000000,0.0,1.0
20998,33065,west_coast_teens,West Coast teenage girls,"F, age 18",en,I was in my backyard and I was flying. I would...,1ANI,AN 1ISA,"1ANI > D, 1ANI > Q, 1ISA 1> Q",NaN,...,0.000000,1.000000,0.000000,0.000000,0.0,1.000,3.000000,0.000000,0.0,1.0
